In [1]:
# install syft if we are on google colab
import sys, os; os.system("pip install --pre syft") if "google.colab" in sys.modules else ""
import syft as sy
print(f"You're running syft version: {sy.__version__}")

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You're running syft version: 0.7.0-beta.18


In [2]:
domain_client = sy.login(email='sam@sg1.net', password='stargate',port = 8081)

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
domain_client.datasets[-1]

Dataset: MedNIST Data 1/10
Description: The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.
Label Count: 6
Label Mapping: {"AbdomenCT": 0, "BreastMRI": 1, "CXR": 2, "ChestCT": 3, "Hand": 4, "HeadCT": 5}
Image Dimensions: (64, 64)
Total Images: 5895




Asset Key,Type,Shape
"[""train_images""]",,"(4719, 4096)"
"[""train_labels""]",,"(4719,)"
"[""val_images""]",,"(601, 4096)"
"[""val_labels""]",,"(601,)"
"[""test_images""]",,"(575, 4096)"
"[""test_labels""]",,"(575,)"


In [4]:
data = domain_client.datasets[-1]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

In [5]:
(X_train, Y_train), (X_dev, Y_dev)

((<TensorPointer -> canada:eb4173ef09d247698d278dc1e0507e61>,
  <TensorPointer -> canada:b7a7f242cd3c4ea0a280ad81a8e3761a>),
 (<TensorPointer -> canada:eb4173ef09d247698d278dc1e0507e61>,
  <TensorPointer -> canada:b7a7f242cd3c4ea0a280ad81a8e3761a>))

In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm

In [7]:
# Preprocess the images

X_train = X_train.T
X_train = X_train / 255.0

X_dev = X_dev.T
X_dev = X_dev / 255.0

In [20]:
from syft.core.smpc.approximations import exp
def init_params(domain_client):
    W1 = np.random.rand(6, 4096) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    W1 = W1.send(domain_client)
    b1 = b1.send(domain_client)
    W2 = W2.send(domain_client)
    b2 = b2.send(domain_client)
    return W1, b1, W2, b2

def ReLU(Z):
    return Z*(Z>0)

def softmax(Z):
    A = exp(Z) / exp(Z).sum()
    return A
    
def forward_prop(W1, b1, W2, b2, X):    
    Z1 = W1@(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2@(A1) + b2
    A2 = softmax(Z2) 
    return Z1, A1, Z2, A2


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2@(A1.T)
    db2 = 1 / m * dZ2.sum()
    dZ1 = W2.T@(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1@(X.T)
    db1 = 1 / m * dZ1.sum()
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [21]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params(domain_client)
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        # dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        # W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 40 == 0:
        #     print("Predicition will be availabe in the next iteration.....Thank you for your support.")
        #     # predictions = get_predictions(A2)
        #     # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [22]:
X_train.public_shape = (4096, 4731)
X_train.public_dtype = np.int64()
Y_train.public_shape = (4731,)


In [23]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

[INFO]: Starting training!



  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Invalid Type for TensorPointer:<class 'syft.proxy.numpy.ndarrayPointer'>

In [ ]:
v = init_params()

In [ ]:
X_train@v[0]

In [ ]:
X_train.shape

In [ ]:
v[0]@X_train